In [ ]:
# import snf
import numpy as np
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import spectral_clustering
import snf

### Load data & preprocessing

In [ ]:
sns.set(style="white", font='Arial', font_scale=1.)
sns.set_palette('rainbow', 4)

In [ ]:
infile = f'results/edges_scores.csv'

In [ ]:
labels   = pd.read_csv('data/ADNI/ADNIMERGE_processed.csv', index_col='PTID')
labels   = labels.loc[labels['PTETHCAT'] == 'Not Hisp/Latino']
labels   = labels.loc[labels['PTRACCAT'] == 'White']

data_dis = pd.read_csv(infile, index_col=0).T
data_dis = pd.concat([labels[['DX', 'PTETHCAT', 'PTRACCAT']], data_dis], axis=1, join='inner')
print(data_dis.shape)

x = data_dis.dropna()
true_labels = x['DX']
x = x.drop(columns=['DX', 'PTETHCAT', 'PTRACCAT'])
# x = x.dropna(axis=1)

In [ ]:
scaler = StandardScaler()
values = scaler.fit_transform(x)
x_scal = pd.DataFrame(values, index=x.index, columns=x.columns)
print('Input data:', x_scal.shape)

In [ ]:
true_labels.value_counts()

### Similarity Network Fusion + Spectral Clustering

In [ ]:
gen = x_scal.values

In [ ]:
si_scores = []; ch_scores = []; db_scores = []
for k in range(2, 11):
    
    affinity_network = snf.make_affinity(gen, metric='sqeuclidean', K=70, mu=0.3)
    labels = spectral_clustering(affinity_network, n_clusters=k)
    
    si = round(snf.metrics.silhouette_score(affinity_network, labels), 4)   
    si_scores.append(si)
    
    print(k, si)

plt.figure(figsize=(5, 4))
sns.lineplot(x=range(2, 11), y=si_scores, markers=True, marker='o')
plt.xticks(range(2, 11))
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Index score')
# plt.tight_layout()
# plt.savefig('figures/SI_scores.pdf', dpi=300)

In [ ]:
affinity_networks = snf.make_affinity(gen, metric='sqeuclidean', K=70, mu=0.3) 
labels_best   = spectral_clustering(affinity_networks, n_clusters=2)
labels_second = spectral_clustering(affinity_networks, n_clusters=3)

In [ ]:
results = x.copy()
results['cluster_2'] = labels_best
results['cluster_3'] = labels_second

print(results['cluster_2'].value_counts())
print()
print(results['cluster_3'].value_counts())

results.to_csv('results/results_clustering.csv')